<a href="https://colab.research.google.com/github/rbdus0715/Machine-Learning/blob/main/study/cuda/01.intro-cuda/04_organization_of_threads_blockIdx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **(1) 구조 파악**

- grid 내부의 모든 스레드는 같은 커널 함수를 수행한다.
- 하나의 블록 안에 존재하는 모든 스레드는 동일한 블록 인덱스를 공유하고 있고, 이 인덱스 값은 커널 안에서 **blockIdx** 변수의 값이다.
- 또한 각 스레드들은 블록 안에서의 위치를 파악하기 위한 스레드 인덱스를 갖는데 이것을 **threadIdx** 변수라고 한다.

</br>

**[예시1]**
1차원 grid, 1 스레드 블록 안에 8개의 스레드

|A|B|C|D|E|F|G|H|
|-|-|-|-|-|-|-|-|

- threadIdx.x : 0 1 2 3 4 5 6 7
- threadIdx.y : 0 0 0 0 0 0 0 0
- threadIdx.z : 0 0 0 0 0 0 0 0

</br>

**[예시2]**
1차원 grid, 2 스레드 블록 안에 4개의 스레드

|A|B|C|D|
|-|-|-|-|

|E|F|G|H|
|-|-|-|-|

두 개의 스레드 블록이므로 서로 따로 인덱싱
- threadIdx.x : 0 1 2 3   0 1 2 3

1차원이기 때문에 y와 z는 0으로 초기화
- threadIdx.y : 0 0 0 0   0 0 0 0
- threadIdx.z : 0 0 0 0   0 0 0 0

</br>

**[예시3]**
2차원 grid, (2 thread block, 2 thread block), 각각의 스레드 블록에는 X 차원에 대해 4개의 스레드가 있음 (스레드 블럭은 1차원)

|P||Q||
|-|-|-|-|

||R||S|
|-|-|-|-|

</br>

|T||U||
|-|-|-|-|

|V|||X|
|-|-|-|-|

-------------------- P Q R S ---- T U V X  ------------</br>
threadIdx.x : 0 2 1 3 ---- 0 2 0 3</br>
threadIdx.y : 0 0 0 0 ---- 0 0 0 0</br>
threadIdx.z : 0 0 0 0 ---- 0 0 0 0

</br>

**[예시4]**
- 이번에는 thread block도 2D로 이루어져있다.
- 2D grid, (2 * 2) 크기, 각 thread block도 (2 * 4) 크기의 thread로 이루어져있다.

|_|X|_|_|
|-|-|-|-|
|_|Y|_|_|

</br>

|P|_|_|_|
|-|-|-|-|
|_|_|Q|_|

</br>

|R|_|_|_|
|-|-|-|-|
|_|_|_|S|

</br>

|_|T|_|_|
|-|-|-|-|
|U|_|_|_|

-------------------- X Y P Q ---- R S T U  ------------</br>
threadIdx.x : 1 1 0 2 ---- 0 3 1 0</br>
threadIdx.y : 0 1 0 1 ---- 0 1 0 1</br>
threadIdx.z : 0 0 0 0 ---- 0 0 0 0

In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

### **(2) 실습**
- 스레드의 총 개수 : 256개의 스레드 (16 * 16)
- 구조 :
    - (2 * 2) 2D 그리드
    - 그리드 한 변 : 16스레드
    - 그리드는 4개의 스레드 블럭으로 이루어져있다.
    - 스레드 블럭의 한 변은 8스레드 (스레드 블럭당 64개의 스레드)

In [5]:
%%cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>

__global__ void print_threadIdx() {
    printf("threadIdx.x : %d, threadIdx.y : %d, threadIdx.z : %d\n",
           threadIdx.x, threadIdx.y, threadIdx.z);
}


int main() {

    int nx, ny;
    nx = 16;
    ny = 16;

    dim3 block(8, 8);
    dim3 grid(nx/block.x, ny/block.y);

    print_threadIdx<<<grid, block>>>();
    cudaDeviceSynchronize();

    cudaDeviceReset();
    return 0;
}

threadIdx.x : 0, threadIdx.y : 0, threadIdx.z : 0
threadIdx.x : 1, threadIdx.y : 0, threadIdx.z : 0
threadIdx.x : 2, threadIdx.y : 0, threadIdx.z : 0
threadIdx.x : 3, threadIdx.y : 0, threadIdx.z : 0
threadIdx.x : 4, threadIdx.y : 0, threadIdx.z : 0
threadIdx.x : 5, threadIdx.y : 0, threadIdx.z : 0
threadIdx.x : 6, threadIdx.y : 0, threadIdx.z : 0
threadIdx.x : 7, threadIdx.y : 0, threadIdx.z : 0
threadIdx.x : 0, threadIdx.y : 1, threadIdx.z : 0
threadIdx.x : 1, threadIdx.y : 1, threadIdx.z : 0
threadIdx.x : 2, threadIdx.y : 1, threadIdx.z : 0
threadIdx.x : 3, threadIdx.y : 1, threadIdx.z : 0
threadIdx.x : 4, threadIdx.y : 1, threadIdx.z : 0
threadIdx.x : 5, threadIdx.y : 1, threadIdx.z : 0
threadIdx.x : 6, threadIdx.y : 1, threadIdx.z : 0
threadIdx.x : 7, threadIdx.y : 1, threadIdx.z : 0
threadIdx.x : 0, threadIdx.y : 2, threadIdx.z : 0
threadIdx.x : 1, threadIdx.y : 2, threadIdx.z : 0
threadIdx.x : 2, threadIdx.y : 2, threadIdx.z : 0
threadIdx.x : 3, threadIdx.y : 2, threadIdx.z : 0
